# 流式传输

<img src="./assets/LC_streaming.png" width="400">

流式传输可以减少数据生成与用户接收之间的延迟。
在 Agent 中常用两种类型：

## 设置

加载并/或检查所需的环境变量

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# 从 .env 加载环境变量
load_dotenv()

# 检查并打印结果
doublecheck_env(".env")

DASHSCOPE_API_KEY=****0fbe
DASHSCOPE_BASE_URL=****e/v1
LANGSMITH_API_KEY=****5ced
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain.agents import create_agent
from langchain_qwq import ChatQwen
import os
llm=ChatQwen(
    model="qwen3-max",
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    api_key=os.getenv("DASHSCOPE_API_KEY")
)

In [3]:
agent = create_agent(
    model=llm,
    system_prompt="You are a full-stack comedian",
)

## 不使用流式（invoke）

In [4]:
result = agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]})
print(result["messages"][1].content)

I’m a full‑stack comedian: my frontend jokes get laughs, my backend jokes return a 500. 

Want another?


## values 模式
你已经在之前的示例中看到过这种流式模式。

In [6]:
# 流式模式 = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

Why don't skeletons fight each other?

Because they don’t have the guts! 💀

*(leans in with a cheesy grin, then winks)*  
...Get it? *Guts?* Like, organs? But they’re just bones?! 😏


In [7]:
# 流式模式 = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    print(step)
    print("-----")

{'messages': [HumanMessage(content='Tell me a Dad joke', additional_kwargs={}, response_metadata={}, id='ccc6edf4-aac0-494f-a0cc-bf733e609a59')]}
-----
{'messages': [HumanMessage(content='Tell me a Dad joke', additional_kwargs={}, response_metadata={}, id='ccc6edf4-aac0-494f-a0cc-bf733e609a59'), AIMessage(content="Why don't skeletons fight each other?\n\nBecause they don’t have the guts! 💀\n\n*(leans in with a cheesy grin, then mimes pulling out imaginary guts like spaghetti)*  \n...Get it? *Guts?* 😏", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 24, 'total_tokens': 73, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-6b72b4da-4d7c-4cf0-bc2a-68eadd9da296', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--82c793a0-594d-4f7e-a3a5-ba2144a41797-0', usage_metad

In [8]:
# 流式模式 = updates
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="updates",
):
    print(step)
    print("-----")

{'model': {'messages': [AIMessage(content="Why don't skeletons fight each other?\n\nBecause they don’t have the guts! 💀\n\n*(leans in with a cheesy grin, then mimes pulling out imaginary intestines like party streamers)*  \n...Get it? *Guts?* 😏", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 24, 'total_tokens': 76, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-b18ac1df-986c-4ddb-a6a1-8fbc6f191484', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--be9f295b-3202-47e2-8d79-ef34fe1d8bbe-0', usage_metadata={'input_tokens': 24, 'output_tokens': 52, 'total_tokens': 76, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})]}}
-----


## messages 模式
消息会逐个 token 流式传输，延迟降到最低，非常适合聊天机器人等交互式应用。

In [5]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "Write me a family friendly poem."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

Here's a cheerful, family-friendly poem celebrating everyday magic:

**The Cookie Jar Secret**

In Grandma’s kitchen, warm and bright,  
Where sunbeams dance with pure delight,  
Stands a jar upon the shelf—  
Holding more than cookies itself!  

It’s not just flour, sugar, spice,  
Or chocolate chips that taste so nice.  
It’s **laughter** mixed in every batch,  
And **patience** (though the timer’s watch!).  

It’s **sticky fingers**, side by side,  
While little helpers take their pride  
In stirring batter, slow and grand—  
A messy, sweet, collaborative hand!  

It’s **Grandpa sneaking** one or two  
(We see you, Pop! We’re watching you!),  
His wink says, "Shh! Our secret’s safe,"  
As crumbs escape his apron’s drape.  

It’s **sisters sharing** sprinkles bright,  
Turning dough into pure light.  
It’s **brothers licking** spoons so clean,  
The best reward they’ve ever seen!  

So when the oven *dings!* so sweet,  
And golden treats can’t be beat,  
We gather ‘round, both young 

In [4]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "你好"}]},
    stream_mode="messages",
):
    print(token)
    print(type(token))
    #print(metadata)
    print("-----")

content='' additional_kwargs={} response_metadata={'model_provider': 'dashscope'} id='lc_run--940e52bf-2f98-4baf-b722-488090643672'
<class 'langchain_core.messages.ai.AIMessageChunk'>
-----
content='哎' additional_kwargs={} response_metadata={'model_provider': 'dashscope'} id='lc_run--940e52bf-2f98-4baf-b722-488090643672'
<class 'langchain_core.messages.ai.AIMessageChunk'>
-----
content='哟' additional_kwargs={} response_metadata={'model_provider': 'dashscope'} id='lc_run--940e52bf-2f98-4baf-b722-488090643672'
<class 'langchain_core.messages.ai.AIMessageChunk'>
-----
content='！' additional_kwargs={} response_metadata={'model_provider': 'dashscope'} id='lc_run--940e52bf-2f98-4baf-b722-488090643672'
<class 'langchain_core.messages.ai.AIMessageChunk'>
-----
content='可' additional_kwargs={} response_metadata={'model_provider': 'dashscope'} id='lc_run--940e52bf-2f98-4baf-b722-488090643672'
<class 'langchain_core.messages.ai.AIMessageChunk'>
-----
content='算等到你了' additional_kwargs={} response_

## 工具也可以流式！
流式通常意味着在最终结果准备好之前就把信息传递给用户，这在很多场景中很有用。`get_stream_writer` 允许你轻松地从自己创建的来源中流式发送 `custom` 数据。

In [15]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """获取指定城市的天气。"""
    writer = get_stream_writer()
    # 可流式输出任意自定义数据
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model=llm,
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='0f0b79db-5e22-419f-ab04-128853a7e84d')]})
('values', {'messages': [HumanMessage(content='What is the weather in SF?', additional_kwargs={}, response_metadata={}, id='0f0b79db-5e22-419f-ab04-128853a7e84d'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 259, 'total_tokens': 280, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-38243810-5585-4061-9d82-d5fe89c4f160', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--487037b0-e654-4f07-8a54-cffeec8a24d3-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'SF'}, 'id': 'call_b8ad676b4501444aa52d95cc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 

In [18]:
from langchain.agents import create_agent
# 工具端：在工具中使用 runtime.stream_writer 推送流式更新
from langchain.tools import tool, ToolRuntime

@tool
def get_weather(city: str, runtime: ToolRuntime) -> str:
    """获取指定城市的天气（示例带流式进度）。"""
    writer = runtime.stream_writer           # ToolRuntime 提供的流写入器（隐藏于 LLM）
    writer(f"Looking up data for city: {city}")    # 推送实时更新（会出现在 stream 的 custom channel）
    # ...执行耗时操作（HTTP 请求、爬取等）
    writer(f"Acquired data for city: {city}")      # 再次推送进度
    return f"It's always sunny in {city}!"         # 常规返回（会作为 ToolMessage / Tool 的输出被捕获）


agent = create_agent(
    model=llm,
    tools=[get_weather],
)

# 调用端：同时订阅 values（完整状态）和 custom（工具 writer 输出）
for mode, chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],   # 同时请求两类流
):
    if mode == "values":
        # chunk 是当前完整 state 快照（通常包含 messages 列表）
        print("LATEST:", chunk["messages"][-1])
        print(type(chunk["messages"][-1]))
    else:
        # mode == "custom": chunk 是工具 writer 推送的文本（或自定义结构）
        print("TOOL STREAM:", chunk)
    print("-----")

LATEST: content='What is the weather in SF?' additional_kwargs={} response_metadata={} id='a30e826d-8ab9-4287-a28a-d5ce79617bcd'
<class 'langchain_core.messages.human.HumanMessage'>
-----
LATEST: content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 266, 'total_tokens': 287, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'dashscope', 'model_name': 'qwen3-max', 'system_fingerprint': None, 'id': 'chatcmpl-23cb42ff-3d21-4a81-8ec5-44e0650307a6', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--b44564e9-3368-46ae-9e5d-46773a801329-0' tool_calls=[{'name': 'get_weather', 'args': {'city': 'SF'}, 'id': 'call_70a8894b68e744fca508db9c', 'type': 'tool_call'}] usage_metadata={'input_tokens': 266, 'output_tokens': 21, 'total_tokens': 287, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}}
<class 'langchain_core.messages.ai.AI

In [8]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: San Francisco')
('custom', 'Acquired data for city: San Francisco')


## 自己尝试不同模式！
修改流式模式和 select，看看能得到哪些不同结果。

In [ ]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])